<a href="https://colab.research.google.com/github/arasavillihemalatha31/FUTURE_ML_03/blob/main/FUTURE_ML_03_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
import random
import string
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


In [13]:
intents = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hi", "Hello", "Hey", "Good morning", "Good evening"],
            "responses": ["Hello! How can I help you today?", "Hi there, what can I do for you?"]
        },
        {
            "tag": "order_status",
            "patterns": ["Where is my order?", "Order status", "Track my order"],
            "responses": ["Please provide your order ID to check the status."]
        },
        {
            "tag": "goodbye",
            "patterns": ["Bye", "See you later", "Goodbye"],
            "responses": ["Goodbye! Have a great day.", "See you next time."]
        }
    ]
}


In [14]:
# Tokenizer (lowercase + remove punctuation)
def simple_tokenizer(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text.strip().split()

# Prepare data
all_words = []
tags = []
xy = []

for intent in intents["intents"]:
    tag = intent["tag"]
    tags.append(tag)
    for pattern in intent["patterns"]:
        tokens = simple_tokenizer(pattern)
        all_words.extend(tokens)
        xy.append((tokens, tag))

all_words = sorted(set(all_words))
tags = sorted(set(tags))

# Bag of words function
def bag_of_words(tokens, all_words):
    return np.array([1 if w in tokens else 0 for w in all_words])

# Create training data
X_train = []
y_train = []

for (pattern_tokens, tag) in xy:
    bow = bag_of_words(pattern_tokens, all_words)
    X_train.append(bow)
    y_train.append(tags.index(tag))

X_train = np.array(X_train)
y_train = np.array(y_train)


In [15]:
model = Sequential()
model.add(Dense(8, input_shape=(len(X_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(tags), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=200, batch_size=5)


Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.3318 - loss: 1.2020
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5023 - loss: 1.0707
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2364 - loss: 1.4441
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4977 - loss: 0.9470
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1909 - loss: 1.2190
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3114 - loss: 1.1702
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2364 - loss: 1.1392
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2614 - loss: 1.0939
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3614 - loss: 1.3070
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2114 - loss: 1.3314    
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4273 - loss: 0.9230
Epoch 12/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6932 

In [21]:
def chat():
    print("Bot: Hello! Type 'quit' to exit.")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        # Tokenize user input and print tokens
        tokens = simple_tokenizer(inp)
        print("User tokens:", tokens)  # Debug: see tokenized user input

        # Convert the tokens to bag of words
        bow = bag_of_words(tokens, all_words)

        # Predict the result
        result = model.predict(np.array([bow]))[0]
        print("Confidence scores:", result)  # Debug: see the confidence for each tag

        idx = np.argmax(result)
        tag = tags[idx]
        confidence = result[idx]

        print("Predicted tag:", tag, "with confidence:", confidence)  # Debug: print predicted tag and confidence

        # Respond based on confidence threshold (lowered to 0.4 for testing)
        if confidence > 0.4:  # Lowered threshold to 0.4
            for tg in intents["intents"]:
                if tg["tag"] == tag:
                    responses = tg["responses"]
            print("Bot:", random.choice(responses))
        else:
            print("Bot: I'm not sure I understand. Can you rephrase?")


In [22]:
chat()


Bot: Hello! Type 'quit' to exit.
User tokens: ['hello']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Confidence scores: [0.2646374  0.44815838 0.28720424]
Predicted tag: greeting with confidence: 0.44815838
Bot: Hello! How can I help you today?
User tokens: ['where', 'is', 'my', 'order']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Confidence scores: [0.11534955 0.22757633 0.65707415]
Predicted tag: order_status with confidence: 0.65707415
Bot: Please provide your order ID to check the status.
User tokens: ['how', 'are', 'you']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Confidence scores: [0.47225574 0.24361834 0.2841259 ]
Predicted tag: goodbye with confidence: 0.47225574
Bot: See you next time.
You: quit
